In [1]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import time

In [2]:
# helper analytics    
def bsPrice(spot, strike, vol, T):
    d1 = (np.log(spot/strike) + 0.5 * vol * vol * T) / vol / np.sqrt(T)
    d2 = d1 - vol * np.sqrt(T)
    return spot * norm.cdf(d1) - strike * norm.cdf(d2)

def bsDelta(spot, strike, vol, T):
    d1 = (np.log(spot/strike) + 0.5 * vol * vol * T) / vol / np.sqrt(T)
    return norm.cdf(d1)

def bsVega(spot, strike, vol, T):
    d1 = (np.log(spot/strike) + 0.5 * vol * vol * T) / vol / np.sqrt(T)
    return spot * np.sqrt(T) * norm.pdf(d1)
#

In [ ]:
def brownian(S0, dt, sigma, mu, z):
    dt_sqrt = tf.math.sqrt(dt)
    shock = sigma * dt_sqrt * z
    drift = (mu - (sigma ** 2) / 2)
    bm = tf.math.exp(drift * dt + shock)
    out = S0 * tf.math.cumprod(bm, axis=1)
    return out

In [47]:
class BlackScholes:
    
    def __init__(self, vol = 0.2, T1 = 1, T2 = 2, K = 1.10, volMult = 1):
        
        self.spot = 1 # S0
        self.vol = vol # sigma
        self.T1 = T1
        self.T2 = T2
        self.K = K
        self.volMult = volMult
    
    def trainingSet(self, m, seed = None):
        
        np.random.seed(seed)
        
        # 2 sets of normal returns
        returns = np.random.normal(size=[m, 2])

        # SDE
        vol0 = self.vol * self.volMult
        R1 = np.exp(-0.5*vol0*vol0*self.T1 + vol0*np.sqrt(self.T1)*returns[:,0])
        R2 = np.exp(-0.5*self.vol*self.vol*(self.T2-self.T1) \
                    + self.vol*np.sqrt(self.T2-self.T1)*returns[:,1])
        S1 = self.spot * R1
        S2 = S1 * R2 

        # payoff
        pay = np.maximum(0, S2 - self.K)
        
        X = S1
        Y = pay

        # differentials
        Z =  np.where(S2 > self.K, R2, 0.0).reshape((-1,1)) 
        
        return X.reshape([-1,1]), Y.reshape([-1,1]), Z.reshape([-1,1])
    
    def testSet(self, lower=0.35, upper=1.65, num=100, seed=None):
        
        spots = np.linspace(lower, upper, num).reshape((-1, 1))
        # compute prices, deltas and vegas
        prices = bsPrice(spots, self.K, self.vol, self.T2 - self.T1).reshape((-1, 1))
        deltas = bsDelta(spots, self.K, self.vol, self.T2 - self.T1).reshape((-1, 1))
        vegas = bsVega(spots, self.K, self.vol, self.T2 - self.T1).reshape((-1, 1))
        return spots, spots, prices, deltas, vegas   

In [48]:
bs = BlackScholes()

In [49]:
x,y,z = bs.trainingSet(1000, 1234) 
# x = S(T-1)
# y = payoff(T)
# z = Delta(S(T-1))
xTest, xAxis, yTest, dydxTest, vegas = bs.testSet(num=100, seed=1234)